In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

# function from sklearn to load IRIS data
from sklearn.datasets import load_iris

# data science
import numpy as np
import pandas as pd

# for outlier detection
from pyod.models.lof import LOF
from pyod.models.iforest import IForest
from pyod.models.ocsvm import OCSVM
from pyod.models.knn import KNN
from outlier_detection.rocf import ROCF
from outlier_detection.cbof import CBOF

# preprocessing
from sklearn import preprocessing

# for evaluation
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split

# exploratory data analysis
from matplotlib import pyplot as plt
import seaborn as sns

# bayesian hyperparameter optimization
from hyperopt import hp, Trials, fmin, tpe, STATUS_OK
from hyperopt.pyll import scope
from time import time
from tqdm import tqdm

# IRIS Dataset

The IRIS dataset consists of 3 different types of irises’ (Setosa, Versicolour, and Virginica) petal and sepal length, stored in a 150x4 numpy.ndarray. (Source: https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html).

The authors in the paper "A novel outlier cluster detection algorithm without top-n parameter" selected 50 examples from “setosa” class as normal cluster (labelled as 0) and respectively selected 10 objects from “versicolor” and “virginica” classes as two small outlier clusters (labelled as 1).

The input data has 4 dimensions:
1. sepal length (cm)
2. sepal width (cm)
3. petal length (cm)
4. petal width (cm)

## Prepare Dataset

### Load Dataset

In [ ]:
try:
    iris_od = pd.read_csv("../data/iris.csv")
except:
    # load iris data
    X_iris, y_iris = load_iris(return_X_y=True, as_frame=True)
    iris_data = X_iris.copy()
    iris_data['target'] = y_iris

    # extract normal data
    iris_normal = iris_data.loc[iris_data['target'] == 0]

    # extract anomaly data
    iris_anomaly = pd.DataFrame()

    for target in [1, 2]:
        # retrieve class and randomly select 10 objects
        iris_class = iris_data.loc[iris_data['target'] == target].sample(10, random_state=1234)
        # append to anomaly dataframe
        iris_anomaly = iris_anomaly.append(iris_class, ignore_index=True)

    # change all anomaly targets to '1'
    iris_anomaly['target'] = 1

    # combine normal and outlier dataset
    iris_od = iris_normal.append(iris_anomaly, ignore_index=True)
    iris_od = iris_od.sample(frac=1).reset_index(drop=True)
    
    # save csv as randomness in sampling may produce different results
    iris_od.to_csv("../data/iris.csv", index=False)

# retrieve X, y
X_iris = iris_od.iloc[:, 0:4].values
y_iris = iris_od.target

print("Shape of data: ", iris_od.shape)
iris_od.head(3)

### Exploratory Data Analysis

In [ ]:
fig, axs = plt.subplots(2, 2)
fig.tight_layout(pad=2.0)

i = 0
for row in range(2):
    for col in range(2):
        # plot histogram distributions of features
        axs[row, col].hist(iris_od[iris_od.columns[i]], color='darkgray')
        axs[row, col].set_title(iris_od.columns[i])
        i += 1

In [ ]:
iris_od.describe()

## Replicate Parameters Suggested in Paper

The paper attempted the following algorithms and parameters:
1. LOF
    - k = 20 (number of neighbors to use for kneighbors queries)
    - n = 10, 20, 30 (number of outliers)
2. CBOF
    - k = 8
    - alpha = 0.90, 0.80, 0.68
    - "if alpha is set to 90%, we intend to regard clusters which contain 90% of data points as normal clusters, and the others are abnormal clusters."
3. ROCF
    - k = 9

In [ ]:
# prepare model functions
n_samples = len(iris_od)

# dictionary of models & parameters to test
# LOF, CBOF, ROCF are replications of Table 4 of the paper
functions_dict = {
    'LOF (k=20, n=10)': { 'algo': 'LOF', 'k':20, 'n':10, 'f':LOF(n_neighbors=20, contamination=10/n_samples) }, 
    'LOF (k=20, n=20)': { 'algo': 'LOF', 'k':20, 'n':20, 'f':LOF(n_neighbors=20, contamination=20/n_samples) },
    'LOF (k=20, n=30)': { 'algo': 'LOF', 'k':20, 'n':30, 'f':LOF(n_neighbors=20, contamination=30/n_samples) },
    'CBOF (k=8, alpha=0.9)': {'algo': 'CBOF', 'k': 8, 'n':7, 'f':CBOF(k=8, contamination=0.1) },
    'CBOF (k=8, alpha=0.8)': {'algo': 'CBOF', 'k': 8, 'n':14, 'f':CBOF(k=8, contamination=0.2) },
    'CBOF (k=8, alpha=0.68)': {'algo': 'CBOF', 'k': 8, 'n':23, 'f':CBOF(k=8, contamination=0.32) },
    'ROCF (k=9)': { 'algo': 'ROCF', 'k':9, 'n':None, 'f':ROCF(distance_metric="euclidean", k=9) }
}

In [ ]:
# create output dataframe
iris_results = pd.DataFrame(columns=['algo', 'k', 'n', 'outlier_rate', 'recall', 'precision', 'f1'])

for name, f_dict in functions_dict.items():
    # initialise classifier
    clf = f_dict['f']

    # fit classifier on data
    clf.fit(X_iris)

    # retrieve outliers on train set
    try:
        y_pred = clf.get_outliers()
    except:
        y_pred = clf.labels_


    # derive evaluation metrics
    report = classification_report(y_true=y_iris, y_pred=y_pred, output_dict=True)['1']

    row = { 
        'algo': f_dict['algo'], 'k': f_dict['k'], 'n': f_dict['n'],
        'precision': report['precision'], 'recall': report['recall'], 'f1': report['f1-score']
    }

    # retrieve outlier rate
    try:
        row['outlier_rate'] = clf.get_outlier_rate()
    except:
        row['outlier_rate'] = clf.contamination

    iris_results = iris_results.append(row, ignore_index=True)

In [ ]:
# print results
iris_results.sort_values(by=['f1'], ascending=False)
iris_results

We observe from the table above that the results we obtained are not consistent with those in the paper. This could be due to reasons like (1) different subset of data (anomalies are randomly sampled), (2) different methods of preprocessing, (3) different methods of feature engineering.

## Hyperparameter Tuning

Besides the 3 methods compared in the paper (LOF, CBOF and ROCF), we also tested other outlier detection algorithms, namely K-Nearest Neighbors, Isolation Forest, and One-Class SVM.

For these 6 methods, we conducted an in-sample hyperparameter tuning, to find the optimal parameters for each of the outlier detection algorithms for our IRIS dataset. We used Bayesian hyperparameter optimisation, which uses Bayes Theorem to direct the hyperparameter search in order to find the minimum or maximum of an objective function.

The contamination factor was set at 20/70 for all algorithms, which is the true contamination factor.

In a later section of this notebook, we will show how the lack of knowledge on the contamination factor will affect the outlier detection results.

### Bayesian Hyperparameter Optimisation Function

In [ ]:
def hyperopt(param_space, X, y, num_eval, classifier):  
    '''
    Function that performs Bayesian hyperparameter optimisation 
    to find the optimal parameters for the outlier detection algorithm.
    
    Inputs:
        param_space (dict): A dictionary of the parameters and corresponding space to search.
        X (array): Features of the dataset.
        y (array): Labels of the dataset (0 = normal; 1 = anomaly).
        
        num_eval (int): Number of evaluation rounds.
        classifier (pyOD Object): Outlier detection algorithm.
        
    Outputs:
        trials
        -min(loss) (float): Best in-sample F1 score.
        best_param_values (dict): Dictionary of the best parameters for the classifier.
    '''
    
    start = time()
    
    def objective_function(params):
        # initialise classifier
        clf = classifier(**params)
        # fit data
        clf.fit(X)
        # predict
        try:
            y_pred = clf.labels_
        except: # ROCF algorithm
            y_pred = clf.get_outliers()
        # get F1 score
        score = f1_score(y_true=y, y_pred=y_pred)
        report = classification_report(y_true=y, y_pred=y_pred, output_dict=True)['1']
        # objective is to maximize F1 i.e. minimize -F1
        return {'loss': -report['f1-score'], 'status': STATUS_OK, 'precision': report['precision'], 
               'recall': report['recall']}
    
    trials = Trials()
    
    # minimise objective function
    best_param = fmin(objective_function, param_space, algo=tpe.suggest, max_evals=num_eval, 
                      trials=trials, rstate= np.random.RandomState(1))
    
    loss = [x['result']['loss'] for x in trials.trials] 
    precision = [x['result']['precision'] for x in trials.trials] 
    recall = [x['result']['recall'] for x in trials.trials] 
    
    best_ind = loss.index(min(loss))
    
    best_param_values = best_param
    
    return trials, -loss[best_ind], best_param_values, precision[best_ind], recall[best_ind]

In [ ]:
# create dict to store hyperopt inputs for each algorithm
hyperopt_inputs = dict()

### Local Outlier Factor (LOF)

Reference: https://pyod.readthedocs.io/en/latest/pyod.models.html?#module-pyod.models.lof

Parameters:
1. `n_neighbors` (default=20): Number of neighbors to use by default for kneighbors queries.
2. `algorithm` (default='auto'): Algorithm used to compute the nearest neighbors.
3. `leaf_size` (default=30): Leaf size passed to BallTree or KDTree.

In [ ]:
# define parameter search range
LOF_param_hyperopt = {
    'n_neighbors': scope.int(hp.quniform('n_neighbors', 8, 25, 1)), # might want to decrease upper limit
    'algorithm': hp.choice('algorithm', ['ball_tree', 'kd_tree', 'brute']),
    'leaf_size': scope.int(hp.quniform('leaf_size', 5, 15, 1)),
    'contamination': 20/70, # set to actual outlier % 
}

# num_eval proportional to number of combinations of parameter values for different models
# num_eval = 3**(num_params_to_tune)
LOF_inputs = {'classifier': LOF, 'param_space': LOF_param_hyperopt, 'num_eval': 3**3}
hyperopt_inputs['LOF'] = LOF_inputs

### Cluster-Based Outlier Factor (CBOF)

Reference: https://github.com/ValaryLim/pyODPlus/blob/main/outlier_detection/cbof.py

Parameters:
1. `k` (default=5): k number of nearest neigbours to compute Local Outlier Factor and Local Reachability Density.
2. `lofub` (default=2.0): Threshold set for any point to be considered a core point in a cluster. If LOF(p) <= lofub, p is a core point. 
3. `pct` (default=0.5): Value in range (0, 1]. Percentage to consider if points are local density reachable from one another.

In [ ]:
# define parameter search range
CBOF_param_hyperopt = {
    'k': scope.int(hp.quniform('n_neighbors', 8, 25, 1)),
    'lofub': hp.uniform('lofub', 0.5, 3.0),
    'pct': hp.uniform('pct', 0.2, 0.8),
    'contamination': 20/70, # set to actual outlier % 
}

CBOF_inputs = {'classifier': CBOF, 'param_space': CBOF_param_hyperopt, 'num_eval': 3**3}
hyperopt_inputs['CBOF'] = CBOF_inputs

### Relative Outlier Cluster Factor (ROCF)

Reference: https://github.com/ValaryLim/pyODPlus/blob/main/outlier_detection/rocf.py

Parameters:
1. `k` (default=3): k number of nearest neigbours used to form MUtual Neighbour Graph.

In [ ]:
ROCF_param_hyperopt = {
    'k': scope.int(hp.quniform('n_neighbors', 15, 25, 1)),
}

ROCF_inputs = {'classifier': ROCF, 'param_space': ROCF_param_hyperopt, 'num_eval': 3**1}
hyperopt_inputs['ROCF'] = ROCF_inputs

### k-Nearest Neighbors

Reference: https://pyod.readthedocs.io/en/latest/pyod.models.html?#module-pyod.models.knn

Parameters:
1. `n_neighbors` (default=5): Number of neighbors to use by default for k neighbors queries.
2. `method` (default='largest'): `largest` uses the distance to the kth neighbor as the outlier score, `mean` uses the average of all k neighbors as the outlier score, `median` uses the median of the distance to k neighbors as the outlier score.

In [ ]:
KNN_param_hyperopt = {
    'contamination': 20/70,
    'n_neighbors': scope.int(hp.quniform('n_neighbors', 8, 25, 1)),
    'method': hp.choice('method', ['largest', 'mean', 'median']),
}

KNN_inputs = {'classifier': KNN, 'param_space': KNN_param_hyperopt, 'num_eval': 3**2}
hyperopt_inputs['KNN'] = KNN_inputs

### Isolation Forest (IForest)

Reference: https://pyod.readthedocs.io/en/latest/pyod.models.html?#module-pyod.models.iforest

Parameters:
1. `n_estimators` (default=100): The number of base estimators in the ensemble.
2. `max_samples` (default='auto' which is min(256, n_samples)): The number of samples to draw from X to train each base estimator.
3. `max_features` (default=1): The number of features to draw from X to train each base estimator.

In [ ]:
IF_param_hyperopt = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 1, 50, 1)),
    'max_samples': scope.int(hp.quniform('max_samples', 10, 35, 1)),    
    'contamination': 20/70,
    'max_features': scope.int(hp.quniform('max_features', 1, 4, 1)),    
}

IF_inputs = {'classifier': IForest, 'param_space': IF_param_hyperopt, 'num_eval': 3**3}
hyperopt_inputs['IForest'] = IF_inputs

### One-Class Support Vector Machine (OCSVM)

Reference: https://pyod.readthedocs.io/en/latest/pyod.models.html?#module-pyod.models.ocsvm

Parameters:
1. `kernel` (default='rbf'): Specifies the kernel type to be used in the algorithm. 
2. `nu` (default=0.5):  An upper bound on the fraction of training errors and a lower bound of the fraction of support vectors.

In [ ]:
OCSVM_param_hyperopt = {
    'kernel': hp.choice('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),
    'nu': hp.uniform('nu', 0.4, 0.8),
    'contamination': 20/70,
}

OCSVM_inputs = {'classifier': OCSVM, 'param_space': OCSVM_param_hyperopt, 'num_eval': 3**2}
hyperopt_inputs['OCSVM'] = OCSVM_inputs

### Comparison of Algorithms

In [ ]:
iris_results_tuned = pd.DataFrame(columns=['algo', 'recall', 'precision', 'f1'])

for algo, algo_inputs in hyperopt_inputs.items():
    # run hyperopt
    algo_hyperopt = hyperopt(algo_inputs['param_space'], \
                             X_iris, y_iris, \
                             algo_inputs['num_eval'], algo_inputs['classifier'])
    # retrieve best parameters
    algo_opt = algo_hyperopt[2]
    algo_opt['f1'] = algo_hyperopt[1] # add f1 score
    algo_opt['precision'] = algo_hyperopt[3]
    algo_opt['recall'] = algo_hyperopt[4]
    algo_opt['algo'] = algo # add algo name
    # add to results dataframe
    iris_results_tuned = iris_results_tuned.append(algo_opt, ignore_index=True)

In [ ]:
iris_results_tuned.sort_values(by=['f1'], ascending=False)

In [ ]:
iris_results_tuned[['algo', 'recall', 'precision', 'f1']]

From the table above, we observe that with the right contamination factor of 20/70 set in the algorithms, IForest, OCSVM, and KNN were able to perfectly identify all the outliers with tuning of the other hyperparameters.

ROCF, which does not require the top-n parameter, also managed to identify all outliers perfectly with a n-neighbors of 22.

## Effect of "Top-n" parameter

In the above analysis, we have fixed the outlier rate to 20/70=0.2857 as we know the outlier rate in the IRIS dataset sampled. In this section, we will demonstrate how different values of outlier rate (pre-specified) will affect LOF, CBOF, KNN, IForest, and OCSVM.

This supports the key point of the paper - that the lack of knowledge on the outlier rate will affect the performance of the algorithm hence, the ability to detect outliers in data.

In [ ]:
top_n_results = pd.DataFrame(columns=['algo', 'outlier_rate', 'recall', 'precision', 'f1'])

outlier_rate_lst = [10/70, 20/70, 30/70]
algos_dict = {'LOF': LOF, 'CBOF': CBOF, 'ROCF': ROCF, 'KNN': KNN, 'IForest': IForest, 'OCSVM': OCSVM}

# for each algorithm
for algo_name, f in algos_dict.items():
    
    # for each outlier rate
    for outlier_rate in outlier_rate_lst:
        
        outlier_rate_add = outlier_rate
        
        # get best parameters from tuning
        algo_params = iris_results_tuned.loc[iris_results_tuned['algo'] == algo_name].reset_index().iloc[0]
        
        # initialise classifier
        if algo_name == 'LOF':
            algo_lst = ['ball_tree', 'kd_tree', 'brute']
            clf = LOF(algorithm=algo_lst[int(algo_params['algorithm'])], contamination=outlier_rate,
                      leaf_size=algo_params['leaf_size'], n_neighbors=int(algo_params['n_neighbors']))
            
        elif algo_name == 'KNN':
            method_lst = ['largest', 'mean', 'median']
            clf = KNN(n_neighbors=int(algo_params['n_neighbors']), contamination=outlier_rate, \
                      method=method_lst[int(algo_params['method'])])
            
        elif algo_name == 'IForest':
            clf = IForest(max_features=int(algo_params['max_features']), contamination=outlier_rate, \
                          max_samples=int(algo_params['max_samples']), n_estimators=int(algo_params['n_estimators']))
            
        elif algo_name == 'CBOF':
            clf = CBOF(k=int(algo_params['n_neighbors']), contamination=outlier_rate, \
                       lofub=algo_params['lofub'], pct=algo_params['pct'])
            
        elif algo_name == 'OCSVM':
            kernel_lst = ['linear', 'poly', 'rbf', 'sigmoid']
            clf = OCSVM(kernel=kernel_lst[int(algo_params['kernel'])], \
                        nu=algo_params['nu'], contamination=outlier_rate)
        
        elif algo_name == 'ROCF':
            if outlier_rate != 20/70:
                continue
            else:
                clf = ROCF(k=int(algo_params['n_neighbors']))
            
        # fit classifier on data
        clf.fit(X_iris)
        
        # retrieve predictions
        try:
            y_pred = clf.get_outliers()
        except:
            y_pred = clf.labels_
            
        if algo_name == 'ROCF':
            outlier_rate_add = clf.get_outlier_rate() # get outlier rate calculated by rocf algorithm
        
        report = classification_report(y_true=y_iris, y_pred=y_pred, output_dict=True)
        f1 = report['1']['f1-score']
        precision = report['1']['precision']
        recall = report['1']['recall']
        top_n_results = top_n_results.append({'algo': algo_name, 'outlier_rate': outlier_rate_add, 'recall': recall, \
                                              'precision': precision, 'f1': f1}, \
                                             ignore_index=True)

In [ ]:
top_n_results

From the table above, we observe that generally, if our estimate of outlier rate is inaccurate, running the outlier detection algorithms will likely lead to sub-optimal outlier detection. This highlights the usefulness of the ROCF algorithm which does not require this top-n parameter.

## Evaluation of 0.1 ROCF Limit

- Authors in the paper claimed that the best threshold (threshold set for any cluster to be considered an outlier) is fixed at 0.1.
- Each cluster has an ROCF value. If max({ROCF}) < threshold, no cluster is considered as outlier.
- Else, all clusters with smaller size than cluster with max ROCF are tagged as outliers. 

In [ ]:
# define parameter search range
n_neighbors = 22 # optimal value found from hyperparameter tuning
threshold_lst = [x for x in np.arange(0.05, 0.31, 0.05)]

In [ ]:
# create output dataframe
iris_rocf_results = pd.DataFrame(columns=['threshold', 'outlier_rate', 'recall', 'precision', 'f1'])

for threshold in threshold_lst:
    # create ROCF classifier
    clf = ROCF(k=n_neighbors, threshold=threshold)

    # fit classifier on data
    clf.fit(X_iris)

    # retrieve predictions
    y_pred = clf.get_outliers()

    # derive evaluation metrics
    report = classification_report(y_true=y_iris, y_pred=y_pred, output_dict=True)
    f1 = report['1']['f1-score']
    precision = report['1']['precision']
    recall = report['1']['recall']

    # retrieve outlier rate
    outlier_rate = clf.get_outlier_rate()

    iris_rocf_results = iris_rocf_results.append({'threshold': threshold, 'outlier_rate': outlier_rate, \
                                                  'recall': recall, 'precision': precision, 'f1': f1}, ignore_index=True)

In [ ]:
# original dataset, without preprocessing
iris_rocf_results.sort_values(by=['f1'], ascending=False)

The results show that with an optimal value for k, the threshold does not really affect the classification results in the threshold range of 0.05 to 0.3.

In [ ]:
# get max rocf with best k
clf = ROCF(k=22)
clf.fit(X_iris)
print(max(clf.get_rocfs()))

## Specification of Parameter k
Show that specification of parameter k will affect results of ROCF outlier detection, and this parameter is not easy to determine as well (limitation of ROCF, even though ROCF does not require top-n).

In [ ]:
# define parameter search range
n_neighbors_lst = [x for x in range(1, 31)]

In [ ]:
# create output dataframe
iris_rocf_results_k = pd.DataFrame(columns=['k', 'outlier_rate', 'recall', 'precision', 'f1'])

for n in n_neighbors_lst:
    # create ROCF classifier
    clf = ROCF(k=n)

    # fit classifier on data
    clf.fit(X_iris)

    # retrieve predictions
    y_pred = clf.get_outliers()

    # derive evaluation metrics
    report = classification_report(y_true=y_iris, y_pred=y_pred, output_dict=True)
    f1 = report['1']['f1-score']
    precision = report['1']['precision']
    recall = report['1']['recall']

    # retrieve outlier rate
    outlier_rate = clf.get_outlier_rate()

    iris_rocf_results_k = iris_rocf_results_k.append({'k': n, 'outlier_rate': outlier_rate, \
                                                      'recall': recall, 'precision': precision, 'f1': f1}, ignore_index=True)

In [ ]:
iris_rocf_results_k

In [ ]:
# plot graph of F1 score against hyperparameter k
plt.scatter(iris_rocf_results_k['k'], iris_rocf_results_k['f1'], marker='.', color='darkblue')
plt.title('F1 Score against k (IRIS dataset)')
plt.xlabel('k, number of nearest neighbors')
plt.ylabel('F1 Score')

The results show that the value of k matters in terms of classification performance. F1 score changes depending on the value for k, given all other factors are constant.